In [1]:
import tensorflow as tf
import numpy as np
import cv2
from tqdm import tqdm
from random import shuffle
import os
from sklearn.preprocessing import StandardScaler  

#---------------------------------------------
#Settings: definição dos parêmetros da rede:
n_classes = 2                     # A base de dados DogsAndCats tem 2 classes de objetos!!!!
nepochs = 10                      # Número de épocas para o treinamento!!!
batch_size = 32                   # Número de imagens por batch!!!
image_size = 32                   # Todas as imagens devem ser redimensionadas para 32x32 pixels!!!
nchannels = 3                     #
n_input = image_size * image_size * nchannels # Tamanho da entrada!
learning_rate = 1e-3              # Taxa de aprendizado!!!
#TRAIN_DIR = 'S:/UTFPR/2018-2/Reconhecimento de Padrões em Imagens HD/dog-cat/train'
#TEST_DIR =  'S:/UTFPR/2018-2/Reconhecimento de Padrões em Imagens HD/dog-cat/test'
TRAIN_DIR = '../APS02/Dogs vs. Cats/kaggle/train'
TEST_DIR = '../APS02/Dogs vs. Cats/kaggle/test'

In [2]:

#---------------------------------------------
def multilayer_perceptron (n_camada_1, n_camada_2, x):
    #n_camada_1 = 256 # Número de neurônios na camada 1!!!
    #n_camada_2 = 128 # Número de neurônios na camada 2!!!
    
    # Primeira camada da rede:
    i = 1
    W1 = tf.get_variable('w'+str(i), [n_input, n_camada_1], initializer = tf.random_normal_initializer())
    b1 = tf.get_variable('b'+str(i), [n_camada_1], initializer = tf.random_normal_initializer())
    y1 = tf.nn.sigmoid(tf.matmul(x, W1) + b1) #tf.nn.relu() or tf.matmul(x, W1) + b1!!!! 
    
    # Segunda camada da rede:
    i = i+1
    W2 = tf.get_variable('w'+str(i), [n_camada_1, n_camada_2], initializer = tf.random_normal_initializer())
    b2 = tf.get_variable('b'+str(i), [n_camada_2], initializer = tf.random_normal_initializer())
    y2 = tf.nn.sigmoid(tf.matmul(y1, W2) + b2) 
    
    # Última camada da rede:
    i = i+1
    W3 = tf.get_variable('w'+str(i),[n_camada_2, n_classes], initializer = tf.random_normal_initializer())
    b3 = tf.get_variable('b'+str(i),[n_classes], initializer = tf.random_normal_initializer())
    out_layer = tf.matmul(y2, W3) + b3 

    return out_layer

#---------------------------------------------
def create_label(image_name):
    word_label = image_name.split('.')[-3]
    if word_label == 'cat':
        return np.array([1,0])
    elif word_label == 'dog':
        return np.array([0,1])
    else: 
        print ("Esta classe não existe!!!!!")

#---------------------------------------------
def read_dataset (filename):
    X = []
    Y = []
    for img in tqdm(os.listdir(filename)):
        path = os.path.join(filename, img)
        img_data = cv2.imread(path)
        #img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE) #converte para níveis de cinza
        img_data = cv2.resize(img_data, (image_size, image_size)) #deixa com as dimensões definidas
        
        #primeira forma de normalização
        #min_val = np.min(img_data)
        #max_val = np.max(img_data )
        #img_data = (img_data-min_val)/(max_val-min_val)
        #segunda forma de normalização
        #img_data = img_data/255.0    
        
        #cria os vetores de dados e de labels
        X.append(np.array(img_data))
        Y.append(np.array(create_label(img)))
        
    return X,Y

#---------------------------------------------
def next_batch (num, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = data[idx]
    labels_shuffle = labels[idx]
    return data_shuffle, labels_shuffle


In [4]:
# Leitura da base de dados:
X_train,Y_train = read_dataset (TRAIN_DIR)
X_train = np.asarray(X_train).reshape(-1, n_input)
Y_train = np.asarray(Y_train)

X_test,Y_test = read_dataset (TEST_DIR)
X_test = np.asarray(X_test).reshape(-1, n_input)
Y_test = np.asarray(Y_test)  

  0%|          | 0/12500 [00:00<?, ?it/s]


IndexError: list index out of range

In [3]:
#terceira forma de normalização
#ATENÇÃO: para testar, comente todas as formas de normalização da função read_dataset()
scaler = StandardScaler()  
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)  

# Variáveis do tensorflow:
Y = tf.placeholder(tf.float32, [None, n_classes])
X = tf.placeholder(tf.float32, [None, image_size * image_size * nchannels])

Ypred = multilayer_perceptron (256, 128, X) 

/usr/local/programas/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [ ]:
#---------------------------------------------
if __name__ == "__main__":

    # Leitura da base de dados:
    #X_train,Y_train = read_dataset (TRAIN_DIR)
    #X_train = np.asarray(X_train).reshape(-1, n_input)
    #Y_train = np.asarray(Y_train)

    #X_test,Y_test = read_dataset (TEST_DIR)
    #X_test = np.asarray(X_test).reshape(-1, n_input)
    #Y_test = np.asarray(Y_test)

    #terceira forma de normalização
    #ATENÇÃO: para testar, comente todas as formas de normalização da função read_dataset()
    #scaler = StandardScaler()  
    #scaler.fit(X_train)  
    #X_train = scaler.transform(X_train)  
    #X_test = scaler.transform(X_test)  

    # Variáveis do tensorflow:
    #Y = tf.placeholder(tf.float32, [None, n_classes])
    #X = tf.placeholder(tf.float32, [None, image_size * image_size * nchannels])
   
    #Ypred = multilayer_perceptron (X) 
  
    # Funções de custo:
    error1 = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(tf.nn.softmax(Ypred)), reduction_indices = [1]))
    error2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Ypred, labels = Y)) 
    error3 = tf.reduce_mean(tf.reduce_sum(tf.square(tf.nn.softmax(Ypred) - Y), reduction_indices = [1]))
    error = error1
    
    # Funções para minimização de erro: 
    optimizer1 = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(error)
    optimizer2 = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(error)
    optimizer = optimizer2
 
    corr = tf.equal(tf.argmax(Ypred,1),tf.argmax(Y,1))
 
    accuracy = tf.reduce_mean(tf.cast(corr,tf.float32))

    # Inicialização de variáveis:
    init = tf.initialize_all_variables()

    with tf.Session() as sess:
        sess.run(init)

        # Treino:
        for epoch in range(nepochs):
            train_err = 0
            train_acc = 0
            train_batches = 0
            total_batch = int(len(X_train)/batch_size)
            for i in range(total_batch):
                batch_xs, batch_ys = next_batch (batch_size, X_train, Y_train)
                sess.run(optimizer, feed_dict={X: batch_xs, Y: batch_ys})
                err, acc = sess.run([error,accuracy], feed_dict={X: batch_xs, Y: batch_ys})
                train_err += err
                train_acc += acc
                train_batches += 1
            print("Epoch: ", '%2d' % (epoch+1))
            print("  training loss:\t\t{:.6f}".format(train_err/train_batches))
            print("  validation accuracy:\t\t{:.2f} %".format(train_acc/train_batches * 100))

        # Testes:
        test_err = 0
        test_acc = 0
        test_batches = 0
        total_batch = int(len(X_test)/batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = next_batch (batch_size, X_test, Y_test)
            sess.run(optimizer, feed_dict={X: batch_xs, Y: batch_ys})
            err, acc = sess.run([error,accuracy], feed_dict={X: batch_xs, Y: batch_ys})
            test_err += err
            test_acc += acc
            test_batches += 1
        print("Final results:")
        print("  test loss:\t\t\t{:.6f}".format(test_err/test_batches))
        print("  test accuracy:\t\t{:.2f} %".format((test_acc/test_batches)*100))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
('Epoch: ', ' 1')
  training loss:		1.254891
  validation accuracy:		61.76 %
('Epoch: ', ' 2')
  training loss:		0.812838
  validation accuracy:		67.32 %
('Epoch: ', ' 3')
  training loss:		0.674030
  validation accuracy:		70.52 %
('Epoch: ', ' 4')
  training loss:		0.594986
  validation accuracy:		72.87 %
('Epoch: ', ' 5')
  training loss:		0.541673
  validation accuracy:		74.75 %
('Epoch: ', ' 6')
  training loss:		0.491321
  validation accuracy:		77.25 %
('Epoch: ', ' 7')
  training loss:		0.471740
  validation accuracy:		77.90 %


In [ ]:
# Se precisar modificar o Percepetron, precisa rodar essa célula.
# O Jupyter não permite criar de novo ws e bs com mesmo nome.
#tf.reset_default_graph() #Descomente para rodar

Optimizer 1 Error 1
Epoch:   1
  training loss:		1.881264
  validation accuracy:		48.50 %
Epoch:   2
  training loss:		1.677853
  validation accuracy:		49.78 %
Epoch:   3
  training loss:		1.532538
  validation accuracy:		50.17 %
Epoch:   4
  training loss:		1.425723
  validation accuracy:		51.44 %
Epoch:   5
  training loss:		1.354925
  validation accuracy:		51.47 %
Epoch:   6
  training loss:		1.265790
  validation accuracy:		51.97 %
Epoch:   7
  training loss:		1.220608
  validation accuracy:		51.88 %
Epoch:   8
  training loss:		1.200131
  validation accuracy:		51.47 %
Epoch:   9
  training loss:		1.140760
  validation accuracy:		52.38 %
Epoch:  10
  training loss:		1.110857
  validation accuracy:		52.72 %
Final results:
  test loss:			1.121007
  test accuracy:		51.15 %

Optimizer 1 Error 2
Epoch:   1
  training loss:		1.647909
  validation accuracy:		52.02 %
Epoch:   2
  training loss:		1.593837
  validation accuracy:		51.65 %
Epoch:   3
  training loss:		1.471031
  validation accuracy:		53.42 %
Epoch:   4
  training loss:		1.425581
  validation accuracy:		52.68 %
Epoch:   5
  training loss:		1.365321
  validation accuracy:		52.90 %
Epoch:   6
  training loss:		1.329543
  validation accuracy:		52.51 %
Epoch:   7
  training loss:		1.283104
  validation accuracy:		51.92 %
Epoch:   8
  training loss:		1.233297
  validation accuracy:		53.20 %
Epoch:   9
  training loss:		1.191547
  validation accuracy:		53.08 %
Epoch:  10
  training loss:		1.175796
  validation accuracy:		53.02 %
Final results:
  test loss:			1.188918
  test accuracy:		52.15 %

Optimizer 1 Error 3
Epoch:   1
  training loss:		0.846617
  validation accuracy:		49.85 %
Epoch:   2
  training loss:		0.837694
  validation accuracy:		50.39 %
Epoch:   3
  training loss:		0.814092
  validation accuracy:		51.69 %
Epoch:   4
  training loss:		0.815053
  validation accuracy:		51.14 %
Epoch:   5
  training loss:		0.806016
  validation accuracy:		51.47 %
Epoch:   6
  training loss:		0.811737
  validation accuracy:		51.21 %
Epoch:   7
  training loss:		0.800126
  validation accuracy:		51.90 %
Epoch:   8
  training loss:		0.794531
  validation accuracy:		52.09 %
Epoch:   9
  training loss:		0.787382
  validation accuracy:		52.15 %
Epoch:  10
  training loss:		0.786845
  validation accuracy:		52.24 %
Final results:
  test loss:			0.785623
  test accuracy:		51.95 %

Optimizer 2 Error 1
Epoch:   1
  training loss:		0.813724
  validation accuracy:		56.84 %
Epoch:   2
  training loss:		0.664701
  validation accuracy:		59.29 %
Epoch:   3
  training loss:		0.659137
  validation accuracy:		59.96 %
Epoch:   4
  training loss:		0.652205
  validation accuracy:		61.00 %
Epoch:   5
  training loss:		0.648369
  validation accuracy:		61.60 %
Epoch:   6
  training loss:		0.650465
  validation accuracy:		61.08 %
Epoch:   7
  training loss:		0.641353
  validation accuracy:		62.17 %
Epoch:   8
  training loss:		0.645863
  validation accuracy:		62.09 %
Epoch:   9
  training loss:		0.638951
  validation accuracy:		62.90 %
Epoch:  10
  training loss:		0.636826
  validation accuracy:		62.80 %
Final results:
  test loss:			0.657422
  test accuracy:		60.85 %

Optimizer 2 Error 2
Epoch:   1
  training loss:		0.740862
  validation accuracy:		56.33 %
Epoch:   2
  training loss:		0.662209
  validation accuracy:		59.67 %
Epoch:   3
  training loss:		0.655810
  validation accuracy:		60.35 %
Epoch:   4
  training loss:		0.650997
  validation accuracy:		61.06 %
Epoch:   5
  training loss:		0.646508
  validation accuracy:		61.89 %
Epoch:   6
  training loss:		0.649950
  validation accuracy:		61.12 %
Epoch:   7
  training loss:		0.641638
  validation accuracy:		62.33 %
Epoch:   8
  training loss:		0.644665
  validation accuracy:		61.67 %
Epoch:   9
  training loss:		0.642716
  validation accuracy:		62.20 %
Epoch:  10
  training loss:		0.639696
  validation accuracy:		62.65 %
Final results:
  test loss:			0.645239
  test accuracy:		61.80 %

Optimizer 2 Error 3
Epoch:   1
  training loss:		0.558197
  validation accuracy:		58.94 %
Epoch:   2
  training loss:		0.472880
  validation accuracy:		59.74 %
Epoch:   3
  training loss:		0.458055
  validation accuracy:		61.67 %
Epoch:   4
  training loss:		0.462267
  validation accuracy:		61.09 %
Epoch:   5
  training loss:		0.457469
  validation accuracy:		61.60 %
Epoch:   6
  training loss:		0.454209
  validation accuracy:		62.14 %
Epoch:   7
  training loss:		0.454985
  validation accuracy:		62.10 %
Epoch:   8
  training loss:		0.450691
  validation accuracy:		62.47 %
Epoch:   9
  training loss:		0.452379
  validation accuracy:		62.67 %
Epoch:  10
  training loss:		0.450615
  validation accuracy:		62.89 %
Final results:
  test loss:			0.456251
  test accuracy:		61.93 %